In [ ]:

# coding: utf-8

# In[ ]:

import falcon
import sys, os

from fft import fft
from fft_ip import ipFFT, inBufferFFT
from get_vector import get_complex
from time import time

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")

    f_in = [0 for i in range(1024)]
    with open("FFT_in.txt", "r") as file:
        for i in range(1024):
            f_in[i] = float(file.readline())
    start_function = time()
    ans = fft(f_in)
    end_function = time()

    for i in range(1024):
        inBufferFFT[i] = f_in[i]
    
    start_ip = time()
    ipFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFFT.read(0x00) & 0x4) == 0x0:
        continue
    end_ip = time()

    ip_out = get_complex(inBufferFFT)
    
    error_cnt = 0
    for i in range(len(ans)):
        ip_real = round(ip_out[i].real, 7)
        ans_real = round(ans[i].real, 7)
        ip_imag = round(ip_out[i].imag, 7)
        ans_imag = round(ans[i].imag, 7)
        f = complex(ip_real, ip_imag)
        a = complex(ans_real, ans_imag)
        if (f != a):
            print(f"Expect: {ans[i]}, actual: {ip_out[i]}")
            error_cnt += 1
    if (error_cnt != 0):
        print(f"Failed!!, there are {error_cnt+1} errors")
    else:
        print("Passed!!")

    print(f"{'FFT python execution time:':25} {(end_function - start_function):>.10f} sec")
    print(f"{'FFT IP execution time:':25} {(end_ip - start_ip):>.10f} sec")
    print("=================================================")
    print("Exit process")
    
    

